- 전 단계에서 만들어진 임상도 및 토양도 추출데이터와 함께 해당 지역에 대한 과거 산사태 발행여부를 조합하는 과정
- 나무와 땅에 대한 구성 정보는 시간이 지나도 바뀌지 않는다는것을 가정한다. 따라서 지난 9년간 있었던 산사태의 발생여부를 레이블로 사용한다면 산사태가 발생한 행정동에 대한 변수를, 산사태가 발생하는 변수를 탐지하는 모델에 대한 학습에 사용할수 있다. 


In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string


Bad key "text.kerning_factor" on line 4 in
/Users/ohikendoit/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
soil_information = pd.read_csv("soil_information_derived.csv", encoding='utf-8')
forest_information = pd.read_csv("forest_information_derived.csv", encoding='utf-8')

In [3]:
soil_information = soil_information.drop(soil_information.columns[0], axis=1)
soil_information.head(5)

,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,LDMARK_STN,ADM_DR_CD,ADM_DR_NM
0,1,11,799.2,21.8,164.8,2,12.0,3.0,2.0,20.0,2.0,2,L102,3839033.0,고제면
1,1,11,534.8,15.7,301.6,2,12.0,5.0,2.0,20.0,2.0,2,L102,3839032.0,웅양면
2,1,11,663.0,21.7,156.6,2,12.0,5.0,2.0,20.0,2.0,2,L102,3839032.0,웅양면
3,1,11,850.3,9.8,129.4,2,12.0,3.0,2.0,20.0,2.0,2,L102,3839033.0,고제면
4,1,11,832.9,18.9,55.8,2,12.0,3.0,2.0,20.0,2.0,2,L102,3839033.0,고제면


In [4]:
soil_information['ADM_DR_NM'].value_counts()

와룡면     5812
금강송면    5278
산내면     4714
풍산읍     4246
화개면     4037
        ... 
동서금동       1
성북동        1
오동동        1
원평1동       1
삼문동        1
Name: ADM_DR_NM, Length: 583, dtype: int64

In [5]:
forest_information = forest_information.drop(forest_information.columns[0], axis=1)
forest_information.head(5)

,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,DNST_CD,HEIGHT,LDMARK_STN,ADM_DR_CD,ADM_DR_NM
0,0,0,0,91,NaN,NaN,NaN,NaN,J00091,3701133.0,장기면
1,0,0,0,91,NaN,NaN,NaN,NaN,J00091,3701133.0,장기면
2,0,0,0,91,NaN,NaN,NaN,NaN,J00091,NaN,NaN
3,0,0,0,91,NaN,NaN,NaN,NaN,J00091,3701131.0,대송면
4,0,0,0,91,NaN,NaN,NaN,NaN,J00091,3701111.0,구룡포읍


In [6]:
forest_information['ADM_DR_NM'].value_counts()

죽장면     5512
장기면     4873
오천읍     2987
기계면     2776
구룡포읍    2048
연일읍     1731
동해면     1619
기북면     1562
대송면     1244
호미곶면     581
신광면      302
대이동      231
효곡동      188
제철동      167
청림동       42
송도동       15
감포읍        7
자양면        7
강동면        7
양북면        6
현동면        6
보덕동        5
천북면        3
부남면        3
임고면        2
안강읍        1
양학동        1
Name: ADM_DR_NM, dtype: int64

In [7]:
landslide_events = pd.read_csv('../data/landslide_events.csv', encoding='utf-8')
landslide_events

,date,sd,sgg,umd,sum_cnt,sum_hpa
0,20110709,경상남도,밀양시,내일동,1,1.20
1,20110709,경상남도,밀양시,단장면,4,3.70
2,20110709,경상남도,밀양시,무안면,5,4.20
3,20110709,경상남도,밀양시,부북면,6,7.80
4,20110709,경상남도,밀양시,산외면,1,2.00
...,...,...,...,...,...,...
354,20191003,경상북도,포항시 북구,장량동,1,0.02
355,20191003,경상북도,포항시 북구,중앙동,1,0.40
356,20191003,경상북도,포항시 북구,청하면,4,0.93
357,20191003,경상북도,포항시 북구,환여동,1,0.10


In [18]:
soil_information['landslide'] = soil_information.apply(lambda x: 1 if x in landslide_events['umd'] else 0, axis=1)

TypeError: 'method' object is not subscriptable